In [25]:
import pandapower as pp
from pandapower.networks import create_cigre_network_mv
from pandapower.plotting.plotly import pf_res_plotly

net = create_cigre_network_mv(with_der="all")

Transforming the HV network in the CIGRE-MV model to 220kV

In [26]:
# Transforming bus[0] in a 220kV bus
net.bus.at[0, "vn_kv"] = 220.0
# Transforming trafo[0] and trafo[1] in 220 kv --> 20kV busses 
net.trafo.at[0, "vn_hv_kv"] = 220.0
net.trafo.at[1, "vn_hv_kv"] = 220.0
print(f"## Bus 0 specs: \n{net.bus.iloc[0]}")
print(f"## Trafo 0 specs: \n{net.trafo.iloc[0]}")
print(f"## Trafo 1 specs: \n{net.trafo.iloc[1]}")

## Bus 0 specs: 
name                                               Bus 0
vn_kv                                              220.0
type                                                   b
zone                                            CIGRE_MV
in_service                                          True
geo           {"coordinates":[7.0,16.0], "type":"Point"}
Name: 0, dtype: object
## Trafo 0 specs: 
name                       Trafo 0-1
std_type                        None
hv_bus                             0
lv_bus                             1
sn_mva                          25.0
vn_hv_kv                       220.0
vn_lv_kv                        20.0
vk_percent                  12.00107
vkr_percent                     0.16
pfe_kw                           0.0
i0_percent                       0.0
shift_degree                    30.0
tap_side                        None
tap_neutral                      NaN
tap_min                          NaN
tap_max                          NaN
tap_ste

In [27]:
net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,PV 3,3,0.020,0.0,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
1,PV 4,4,0.020,0.0,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
2,PV 5,5,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
3,PV 6,6,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
4,PV 8,8,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
5,PV 9,9,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
6,PV 10,10,0.040,0.0,NaN,NaN,0.040,1.0,False,<NA>,False,NaN,True,PV,True
7,PV 11,11,0.010,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
8,WKA 7,7,1.500,0.0,NaN,NaN,1.500,1.0,False,<NA>,False,NaN,True,WP,True
9,Residential fuel cell 1,5,0.033,0.0,NaN,NaN,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True


Adding 100kV to each PV generator to match the Italian use-case

In [28]:
for i in range(0, 8):
    if net.sgen.at[i, "p_mw"] < 0.1: 
        net.sgen.at[i, "p_mw"] = net.sgen.at[i, "p_mw"] + 0.1 # Adding 0.1 MW to each PV generator 
        net.sgen.at[i, "q_mvar"] = net.sgen.at[i, "p_mw"] * 0.1
# Setting maximum active powert l
# Setting minimum and maximum reactive power limits for all generators (needed for OPF)
# Make all generators controllable if above 0.1 MW
for i in range(0, 13):
    if net.sgen.at[1, "type"] == "PV":
        net.sgen.at[i, "min_p_mw"] = net.sgen.at[i, "p_mw"]
        net.sgen.at[i, "max_p_mw"] = net.sgen.at[i, "p_mw"]
    else:
        net.sgen.at[i, "min_p_mw"] = net.sgen.at[i, "p_mw"] * 0.75
        net.sgen.at[i, "max_p_mw"] = net.sgen.at[i, "p_mw"] * 1.25
    net.sgen.at[i, "min_q_mvar"] = net.sgen.at[i, "q_mvar"] * 0.75
    net.sgen.at[i, "max_q_mvar"] = net.sgen.at[i, "q_mvar"] * 1.25
    if net.sgen.at[i, "p_mw"] >= 0.1:
        net.sgen.at[i, "controllable"] = True
net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source,min_p_mw,max_p_mw
0,PV 3,3,0.120,0.012,0.00900,0.01500,0.020,1.0,True,<NA>,False,NaN,True,PV,True,0.120,0.120
1,PV 4,4,0.120,0.012,0.00900,0.01500,0.020,1.0,True,<NA>,False,NaN,True,PV,True,0.120,0.120
2,PV 5,5,0.130,0.013,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
3,PV 6,6,0.130,0.013,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
4,PV 8,8,0.130,0.013,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
5,PV 9,9,0.130,0.013,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
6,PV 10,10,0.140,0.014,0.01050,0.01750,0.040,1.0,True,<NA>,False,NaN,True,PV,True,0.140,0.140
7,PV 11,11,0.110,0.011,0.00825,0.01375,0.010,1.0,True,<NA>,False,NaN,True,PV,True,0.110,0.110
8,WKA 7,7,1.500,0.000,0.00000,0.00000,1.500,1.0,True,<NA>,False,NaN,True,WP,True,1.500,1.500
9,Residential fuel cell 1,5,0.033,0.000,0.00000,0.00000,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True,0.033,0.033


In [29]:
# Add bus voltage constraints
for i in range(len(net.bus)):
    net.bus.at[i, "min_vm_pu"] = 0.95
    net.bus.at[i, "max_vm_pu"] = 1.05
net.bus

,name,vn_kv,type,zone,in_service,geo,min_vm_pu,max_vm_pu
0,Bus 0,220.0,b,CIGRE_MV,True,"{""coordinates"":[7.0,16.0], ""type"":""Point""}",0.95,1.05
1,Bus 1,20.0,b,CIGRE_MV,True,"{""coordinates"": [4.0, 15.0], ""type"": ""Point""}",0.95,1.05
2,Bus 2,20.0,b,CIGRE_MV,True,"{""coordinates"": [4.0, 13.0], ""type"": ""Point""}",0.95,1.05
3,Bus 3,20.0,b,CIGRE_MV,True,"{""coordinates"": [4.0, 11.0], ""type"": ""Point""}",0.95,1.05
4,Bus 4,20.0,b,CIGRE_MV,True,"{""coordinates"": [2.5, 9.0], ""type"": ""Point""}",0.95,1.05
5,Bus 5,20.0,b,CIGRE_MV,True,"{""coordinates"": [1.0, 7.0], ""type"": ""Point""}",0.95,1.05
6,Bus 6,20.0,b,CIGRE_MV,True,"{""coordinates"": [1.0, 3.0], ""type"": ""Point""}",0.95,1.05
7,Bus 7,20.0,b,CIGRE_MV,True,"{""coordinates"": [8.0, 3.0], ""type"": ""Point""}",0.95,1.05
8,Bus 8,20.0,b,CIGRE_MV,True,"{""coordinates"": [8.0, 5.0], ""type"": ""Point""}",0.95,1.05
9,Bus 9,20.0,b,CIGRE_MV,True,"{""coordinates"": [6.0, 5.0], ""type"": ""Point""}",0.95,1.05


In [30]:
# Setting load scaling in case needed
for i in range(0, 18):
    net.load.at[i, "scaling"] = 1
net.load

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
0,Load R1,1,14.99400,3.044662,0.0,0.0,0.0,0.0,15.300,1.0,True,wye
1,Load R3,3,0.27645,0.069285,0.0,0.0,0.0,0.0,0.285,1.0,True,wye
2,Load R4,4,0.43165,0.108182,0.0,0.0,0.0,0.0,0.445,1.0,True,wye
3,Load R5,5,0.72750,0.182329,0.0,0.0,0.0,0.0,0.750,1.0,True,wye
4,Load R6,6,0.54805,0.137354,0.0,0.0,0.0,0.0,0.565,1.0,True,wye
5,Load R8,8,0.58685,0.147078,0.0,0.0,0.0,0.0,0.605,1.0,True,wye
6,Load R10,10,0.47530,0.119121,0.0,0.0,0.0,0.0,0.490,1.0,True,wye
7,Load R11,11,0.32980,0.082656,0.0,0.0,0.0,0.0,0.340,1.0,True,wye
8,Load R12,12,14.99400,3.044662,0.0,0.0,0.0,0.0,15.300,1.0,True,wye
9,Load R14,14,0.20855,0.052268,0.0,0.0,0.0,0.0,0.215,1.0,True,wye


In [31]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,Line 1-2,CABLE_CIGRE_MV,1,2,2.82,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
1,Line 2-3,CABLE_CIGRE_MV,2,3,4.42,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
2,Line 3-4,CABLE_CIGRE_MV,3,4,0.61,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
3,Line 4-5,CABLE_CIGRE_MV,4,5,0.56,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
4,Line 5-6,CABLE_CIGRE_MV,5,6,1.54,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
5,Line 7-8,CABLE_CIGRE_MV,7,8,1.67,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
6,Line 8-9,CABLE_CIGRE_MV,8,9,0.32,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
7,Line 9-10,CABLE_CIGRE_MV,9,10,0.77,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
8,Line 10-11,CABLE_CIGRE_MV,10,11,0.33,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
9,Line 3-8,CABLE_CIGRE_MV,3,8,1.30,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None


In [32]:
pf_res_plotly(net, auto_open=False, aspectratio=(1.2,1))

There are no Power Flow results. A Newton-Raphson power flow will be executed.


In [33]:
# Modifiy the reactive power setpoint for all the PV generators
for elem in net.sgen.iterrows():
    if str.startswith(elem[1]["name"], "PV"):  
        p_mw = net.sgen.at[elem[0], "p_mw"]
        net.sgen.at[elem[0], "q_mvar"] = -p_mw * 0.4 # Setting reactive power to -40% of active power (absorbing reactive power)
net.sgen
# TODO: we could study how the number of violated busses changes with the number of compromised PVs

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source,min_p_mw,max_p_mw
0,PV 3,3,0.120,-0.048,0.00900,0.01500,0.020,1.0,True,<NA>,False,NaN,True,PV,True,0.120,0.120
1,PV 4,4,0.120,-0.048,0.00900,0.01500,0.020,1.0,True,<NA>,False,NaN,True,PV,True,0.120,0.120
2,PV 5,5,0.130,-0.052,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
3,PV 6,6,0.130,-0.052,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
4,PV 8,8,0.130,-0.052,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
5,PV 9,9,0.130,-0.052,0.00975,0.01625,0.030,1.0,True,<NA>,False,NaN,True,PV,True,0.130,0.130
6,PV 10,10,0.140,-0.056,0.01050,0.01750,0.040,1.0,True,<NA>,False,NaN,True,PV,True,0.140,0.140
7,PV 11,11,0.110,-0.044,0.00825,0.01375,0.010,1.0,True,<NA>,False,NaN,True,PV,True,0.110,0.110
8,WKA 7,7,1.500,0.000,0.00000,0.00000,1.500,1.0,True,<NA>,False,NaN,True,WP,True,1.500,1.500
9,Residential fuel cell 1,5,0.033,0.000,0.00000,0.00000,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True,0.033,0.033


In [34]:
pp.runpp(net)
pf_res_plotly(net, auto_open=False, aspectratio=(1.2,1))
net.res_sgen

,p_mw,q_mvar
0,0.120,-0.048
1,0.120,-0.048
2,0.130,-0.052
3,0.130,-0.052
4,0.130,-0.052
5,0.130,-0.052
6,0.140,-0.056
7,0.110,-0.044
8,1.500,0.000
9,0.033,0.000


In [35]:
pp.runopp(net)
pf_res_plotly(net, auto_open=False, aspectratio=(1.2,1))

no costs are given - overall generated power is minimized


In [39]:
net.res_sgen
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.030000,0.000000,-42.595221,-15.395731,1.000000,7.045086e-20
1,0.995274,-35.877028,19.839000,4.637136,1.004470,1.477973e-02
2,0.981863,-36.257147,0.000000,0.000000,1.030018,3.730160e-02
3,0.960396,-36.852279,0.381700,0.195969,1.071887,7.366325e-02
4,0.958490,-36.964804,0.311650,0.095264,1.076380,7.544779e-02
5,0.957060,-37.052679,1.164500,0.168297,1.079777,7.666209e-02
6,0.955911,-37.112072,0.418050,0.123318,1.082687,7.835027e-02
7,0.962134,-36.482356,-1.423500,0.047410,1.065844,8.030870e-02
8,0.959152,-36.750803,0.456850,0.133035,1.073095,7.957871e-02
9,0.958530,-36.763705,-0.078250,0.341532,1.074215,8.064355e-02
